In [1]:
import fbx

In [30]:
memory_manager = fbx.FbxManager.Create()
scene = fbx.FbxScene.Create(memory_manager, '')

mesh_node = fbx.FbxNode.Create(memory_manager, 'tri')
scene.GetRootNode().AddChild(mesh_node)

# -- mesh
mesh_attribute: fbx.FbxMesh = fbx.FbxMesh.Create(memory_manager, '')
mesh_node.AddNodeAttribute(mesh_attribute)

# -- mesh -- vertices
mesh_attribute.InitControlPoints(3)

TRI_VERTS = (
    (0.0, 0.0, 0.0),
    (0.0, 0.0, 100.0),
    (0.0, 100.0, 0.0)
)

for i in range(0,3):
    vert = TRI_VERTS[i]
    mesh_attribute.SetControlPointAt(fbx.FbxVector4(vert[0], vert[1], vert[2], 0.0), i)
    
# -- mesh -- the face
mesh_attribute.BeginPolygon(0)
mesh_attribute.AddPolygon(2)
mesh_attribute.AddPolygon(1)
mesh_attribute.AddPolygon(0)
mesh_attribute.EndPolygon()

# -- mesh -- normals
        
normal_elem: fbx.FbxLayerElementNormal = mesh_attribute.CreateElementNormal()
normal_elem.SetMappingMode(fbx.FbxLayerElementNormal.eByPolygon)
normal_elem.SetReferenceMode(fbx.FbxLayerElementNormal.eDirect)

normal_elem.GetDirectArray().Add(fbx.FbxVector4(1.0, 0.0, 0.0))

if mesh_attribute.GetLayer(0) is None:
    mesh_attribute.CreateLayer()
layer: fbx.FbxLayer = mesh_attribute.GetLayer(0)

layer.SetNormals(normal_elem)

# -- uvs

uv_element: fbx.FbxLayerElementUV = mesh_attribute.CreateElementUV("UVSet")
uv_element.SetMappingMode(fbx.FbxLayerElement.eByPolygonVertex)
uv_element.SetReferenceMode(fbx.FbxLayerElement.eDirect)

uv_element.GetDirectArray().Add(fbx.FbxVector2(0.0, 0.0))
uv_element.GetDirectArray().Add(fbx.FbxVector2(1.0, 0.0))
uv_element.GetDirectArray().Add(fbx.FbxVector2(0.5, 1.0))

# -- material

# -- material - material element

material_element = mesh_attribute.CreateElementMaterial()
material_element.SetMappingMode(fbx.FbxLayerElement.eByPolygon)
material_element.SetReferenceMode(fbx.FbxLayerElement.eDirect)

# -- material - create material

material = fbx.FbxSurfacePhong.Create(memory_manager, 'material')
material.Diffuse.Set(fbx.FbxDouble3(1.0, 1.0, 1.0))
material.ShadingModel.Set('Phong')
mesh_node.AddMaterial(material)

# -- material - texture

texture = fbx.FbxFileTexture.Create(scene, 'diffuse')
texture.SetFileName('checker.jpg')
texture.SetTextureUse(fbx.FbxTexture.eStandard)
texture.SetMappingType(fbx.FbxTexture.eUV)
texture.SetMaterialUse(fbx.FbxFileTexture.eModelMaterial)
texture.SetSwapUV(False)
texture.SetTranslation(0.0, 0.0)
texture.SetScale(1.0, 1.0)
texture.SetRotation(0.0, 0.0)

material.Diffuse.ConnectSrcObject(texture)

print(material.Diffuse.GetSrcObject().GetFileName())

mesh_node.SetShadingMode(fbx.FbxNode.eTextureShading)

# EXPORT

exporter = fbx.FbxExporter.Create(memory_manager, '')
exporter.Initialize('tri', -1, memory_manager.GetIOSettings())
result = exporter.Export(scene)

if result:
    print('exported succesfully')
    
memory_manager.Destroy()

checker.jpg
exported succesfully


In [17]:
# read proper tri
memory_manager = fbx.FbxManager.Create()

importer = fbx.FbxImporter.Create(memory_manager, '')
importer.Initialize('proper_tri.fbx')

scene = fbx.FbxScene.Create(memory_manager, '')
importer.Import(scene)

tri_node = scene.GetRootNode().GetChild(0)

print('material count: {}'.format(tri_node.GetMaterialCount()))
print('has diffuse texture: {}'.format(tri_node.GetMaterial(0).Diffuse.GetSrcObject() != None))
print('diffuse texture name: {}'.format(tri_node.GetMaterial(0).Diffuse.GetSrcObject().GetFileName()))
print(scene.GetRootNode().GetChild(0).GetName())

memory_manager.Destroy()

material count: 1
has diffuse texture: True
diffuse texture name: E:\projects\the_witcher\content_pipeline\attempts\checker.jpg
tri


In [31]:
# read improper tri
memory_manager = fbx.FbxManager.Create()

importer = fbx.FbxImporter.Create(memory_manager, '')
importer.Initialize('tri.fbx')

scene = fbx.FbxScene.Create(memory_manager, '')
importer.Import(scene)

tri_node = scene.GetRootNode().GetChild(0)

print('node: {}'.format(tri_node.GetName()))
print('material count: {}'.format(tri_node.GetMaterialCount()))
print('has diffuse texture: {}'.format(tri_node.GetMaterial(0).Diffuse.GetSrcObject() != None))
print('diffuse texture name: {}'.format(tri_node.GetMaterial(0).Diffuse.GetSrcObject().GetFileName()))
print(scene.GetRootNode().GetChild(0).GetName())

memory_manager.Destroy()

node: tri
material count: 1
has diffuse texture: True
diffuse texture name: checker.jpg
tri
